In [7]:
from google.colab import drive
drive.mount('/content/drive')

import os
print(os.path.exists('/content/drive/MyDrive/fake_news_model.h5'))
print(os.path.exists('/content/drive/MyDrive/tokenizer.pickle'))

!pip install flask pyngrok

app_code = """
from flask import Flask, request, render_template_string
import tensorflow as tf
import numpy as np
import pickle
import re
import string
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load model and tokenizer
model = tf.keras.models.load_model('/content/drive/MyDrive/fake_news_model.h5')
with open('/content/drive/MyDrive/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Constants
max_length = 50

# Clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[{}]'.format(re.escape(string.punctuation)), ' ', text)
    text = re.sub(r'\\\\w*\\\\d\\\\w*', '', text)
    text = re.sub(r'\\\\s+', ' ', text).strip()
    return text

# Predict function
def predict(text):
    cleaned = clean_text(text)
    seq = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(seq, maxlen=max_length, padding='post', truncating='post')
    prob = model.predict(padded)[0][0]
    label = "Real" if prob > 0.5 else "Fake"
    return label, prob

# Flask setup
app = Flask(__name__)

HTML_TEMPLATE = '''
<!DOCTYPE html>
<html>
  <head><title>Fake News Detection</title></head>
  <body>
    <h2>Enter a News Statement</h2>
    <form method="post">
      <textarea name="news" rows="6" cols="60" placeholder="Enter text here..."></textarea><br><br>
      <input type="submit" value="Check">
    </form>
    {% if result %}
      <p>Confidence: {{ prob }}%</p>
    {% endif %}
  </body>
</html>
'''

@app.route("/", methods=['GET', 'POST'])
def home():
    result = None
    prob = None
    if request.method == 'POST':
        news = request.form['news']
        result, p = predict(news)
        prob = round(p * 100, 2)
    return render_template_string(HTML_TEMPLATE, result=result, prob=prob)

app.run(port=5000)
"""

with open("app.py", "w") as f:
    f.write(app_code)

print("app.py created successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
True
True
✅ app.py created successfully.


In [8]:
from pyngrok import conf, ngrok
import subprocess
import time

conf.get_default().auth_token = "2x7PIQfAxBoe3LxRNrfjYnHVzSK_6Z5VMa2drcaKDVuX17VYy"
public_url = ngrok.connect(5000)
print("Your app is live at:", public_url)
process = subprocess.Popen(["python3", "app.py"])
time.sleep(3)

🚀 Your app is live at: NgrokTunnel: "https://7531-34-48-147-62.ngrok-free.app" -> "http://localhost:5000"
